In [1]:
import spatialdata
import skimage.io as io
import numpy as np
import opendvp
import pandas as pd
import anndata as ad
import geopandas as gpd
from shapely import Polygon
import ast
from typing import Sequence

import anndata as ad
import geopandas as gpd
import numpy as np
import pandas as pd

from opendvp.utils import logger

/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/dev-sdata/lib/python3.13/site-packages/spatialdata/_core/query/relational_query.py:532: FutureWarning: functools.pa

In [2]:
from typing import Literal

import anndata as ad
import numpy as np
import pandas as pd
import pytest

from opendvp.tl.filter_features_byNaNs import filter_features_byNaNs

In [3]:
def create_test_adata(
    n_obs=10,
    n_vars=5,
    nan_pattern : Literal['random', 'structured', 'all_valid', 'all_nan'] | None = None,  
    grouping_col=None,
    var_cols=None
):
    """Creates a dummy AnnData object with specified dimensions and NaN patterns."""
    X = np.random.rand(n_obs, n_vars) * 10

    if nan_pattern == 'random':
        nan_indices = np.random.choice(n_obs * n_vars, int(n_obs * n_vars * 0.2), replace=False)
        X.flat[nan_indices] = np.nan
    elif nan_pattern == 'structured':
        # Introduce structured NaNs for predictable testing
        X[:n_obs//2, 0] = np.nan # Protein 0 has NaNs in first half of samples
        X[n_obs//2:, 1] = np.nan # Protein 1 has NaNs in second half of samples
        X[:, 2] = np.nan # Protein 2 is all NaNs
        X[:, 3] = 1.0 # Protein 3 is all valid
        X[:, 4] = 1.0 # Protein 4 is all valid
    elif nan_pattern == 'all_valid':
        X = np.ones((n_obs, n_vars))
    elif nan_pattern == 'all_nan':
        X = np.full((n_obs, n_vars), np.nan)

    obs = pd.DataFrame(index=[f'cell_{i}' for i in range(n_obs)])
    if grouping_col:
        obs[grouping_col] = ['groupA'] * (n_obs // 2) + ['groupB'] * (n_obs - n_obs // 2)
    
    var_data = {'original_col_1': [f'gene_{i}' for i in range(n_vars)],
                'original_col_2': np.arange(n_vars)}
    if var_cols:
        var_data.update(var_cols)
    var = pd.DataFrame(var_data, index=[f'protein_{i}' for i in range(n_vars)])

    adata = ad.AnnData(X=X, obs=obs, var=var)
    return adata

In [9]:
adata = create_test_adata(n_obs=5, n_vars=5, nan_pattern="random")
adata.X

array([[3.70901514, 1.06302868, 4.67816553, 1.51995282,        nan],
       [9.61504541, 6.79896676, 9.16804934, 8.47162031, 0.06167912],
       [       nan, 0.88580514,        nan, 4.54793265, 3.40813459],
       [3.47695189,        nan, 7.57355187, 7.62349755, 5.94144452],
       [7.58631469, 5.05908974, 1.60172298, 8.08754087,        nan]])

In [14]:
adata = create_test_adata(n_obs=10, n_vars=5, nan_pattern="structured")
adata.X

array([[       nan, 2.17249272,        nan, 1.        , 1.        ],
       [       nan, 1.80508105,        nan, 1.        , 1.        ],
       [       nan, 4.12400906,        nan, 1.        , 1.        ],
       [       nan, 1.19916888,        nan, 1.        , 1.        ],
       [       nan, 7.72565344,        nan, 1.        , 1.        ],
       [8.78656139,        nan,        nan, 1.        , 1.        ],
       [3.10320042,        nan,        nan, 1.        , 1.        ],
       [3.75802253,        nan,        nan, 1.        , 1.        ],
       [9.83712897,        nan,        nan, 1.        , 1.        ],
       [7.30276396,        nan,        nan, 1.        , 1.        ]])

array([[3.75427689,        nan, 7.05459632, 9.79896367,        nan],
       [2.19775071, 8.30591211,        nan, 7.3626556 , 1.97758067],
       [       nan, 0.46552613, 7.12953151, 7.96498521, 9.66215719],
       [7.57196055, 9.47889411, 1.54342085, 1.17498436, 8.41366918],
       [0.70874143,        nan, 9.5174399 , 4.89150773, 9.97833865]])